<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/AI_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI')

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import copy
import json
from openai import OpenAI
#from openai import ChatCompletion

#from utils_local import o1_tools

client = OpenAI(api_key=openai_api_key)
#O1_MODEL = 'o1-mini'
GPT_MODEL = 'gpt-4o-mini'

In [51]:
# System prompt for the execution model
gpt4o_system_prompt = """
You are a helpful data analyst responsible for analysis provided data file using the tools provided.
You must explain your decision-making process across various steps. if ask to read data, you should read the
data file using the tool provided. The data file is located at /content/Beer_small_test.csv
If you are not asked to read the data file, you should answer a user question to the best of your ability.

"""

In [5]:
# Tools
def read_csv_to_dataframe(file_path: str) -> str:
    import pandas as pd
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        return f"Error reading CSV file: {e}"

In [6]:
TOOLS_test = [
    {
        "type": "function",
        "function": {
            "name": "read_csv_to_dataframe",
            "description":"Reads a CSV file and returns the DataFrame",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {"type": "string", "description": "Path to the CSV file."}
                },
                "required": ["file_path"],
                "additionalProperties": False,
            },
        },
    }]

In [ ]:
data = '/content/Beer_small_test.csv'

messages = [
    {"role": "system", "content": 'you are helful data analyst reading the data files'},
    {"role": "user", "content": f"please read the data file"}]

In [68]:
content = '''You are an assistant that uses tools if asked anything that can be addressed by a tool.
              or directly answer user questions as asked
'''
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": gpt4o_system_prompt},
        {"role": "user", "content": f"please read the data file"}
    ],
    tools=TOOLS_test,
    tool_choice="auto"
)

In [66]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_JvES3EhPvvp72lNLgybB6hYS', function=Function(arguments='{"file_path":"/content/Beer_small_test.csv"}', name='read_csv_to_dataframe'), type='function')]

In [37]:
response.choices[0].message.tool_calls[0].function.name

'read_csv_to_dataframe'

In [40]:
response.choices[0].message.tool_calls[0].function.arguments

'{"file_path":"/content/Beer_small_test.csv"}'

In [69]:
if response.choices[0].message.tool_calls:
  # Assuming response.choices[0].message.tool_calls[0] is stored in a variable `tool_call`
  tool_call = response.choices[0].message.tool_calls[0]

  # Extract the function name and arguments
  function_name = tool_call.function.name  # Access the function name
  function_arguments = tool_call.function.arguments  # Access the function arguments

  # Parse the JSON string
  parsed_arguments = json.loads(function_arguments)

  print("Function Name:", function_name)
  print("Function Arguments:", function_arguments)

  file_path = parsed_arguments.get("file_path")

  print("File Path:", file_path)
else:
  print(f'model response: {response.choices[0].message.content}')

In [70]:
response.choices[0].message.content

'The meaning of life is a philosophical question that has been pondered for centuries. Different cultures, religions, and individuals have various interpretations and beliefs regarding this topic. Some people find meaning through religion or spirituality, others through relationships and connections with family and friends, and some find purpose in pursuits of knowledge, creativity, or contribution to society. Ultimately, the meaning of life can be deeply personal and subjective, and what is meaningful to one person may not be the same for another.'

In [102]:
response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_FaeIdLZf0Q6cxFXr18qIy1NG', function=Function(arguments='{"file_path":"data.csv"}', name='read_csv_to_dataframe'), type='function')]))

In [31]:
# Assuming response.choices[0] is stored in a variable called `choice`
choice = response.choices[0]

# Extract the tool_calls list
tool_calls = choice.message.tool_calls

# Extract the first tool call (assuming there's only one call)
if tool_calls and len(tool_calls) > 0:
    tool_call = tool_calls[0]
    function_name = tool_call.function_call["name"]  # Extract the function name
    function_arguments = tool_call.function_call["arguments"]  # Extract the function arguments
    print("Function Name:", function_name)
    print("Function Arguments:", function_arguments)
else:
    print("No tool calls found in the response.")

AttributeError: 'ChatCompletionMessageToolCall' object has no attribute 'function_call'

In [ ]:
if tool_name:
  df = tool_name(tool_args)
  print(type(df))

In [ ]:
#test block 2
import json
import pandas as pd

# Define the tool for reading the CSV
tools = [
    {
        "name": "read_csv_to_dataframe",
        "description": "Reads a CSV file and extracts the requested column data.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {"type": "string", "description": "Path to the CSV file to read."},
                "column_name": {"type": "string", "description": "Name of the column to extract."},
                "num_entries": {"type": "integer", "description": "Number of entries to retrieve from the column."}
            },
            "required": ["file_path", "column_name", "num_entries"]
        }
    }
]

# Define the function to read the CSV and extract data
def read_csv_to_dataframe(file_path: str, column_name: str, num_entries: int):
    try:
        df = pd.read_csv(file_path)
        if column_name not in df.columns:
            return {"error": f"Column '{column_name}' not found in the file."}
        return {
            "data": df[column_name].head(num_entries).tolist(),
            "column_name": column_name
        }
    except Exception as e:
        return {"error": str(e)}

# Define the initial query
messages = [
    {"role": "system", "content": "You are an assistant that can process CSV files and analyze data."},
    {"role": "user", "content": "Please summarize the first 5 entries in the 'text' column of the file '/content/Beer_small_test.csv'."}
]

# Simulate the GPT-4o response (mock client)
class MockGPT4oClient:
    def chat(self, completions):
        # Simulate the GPT-4o model suggesting a function call
        return {
            "choices": [
                {
                    "message": {
                        "function_call": {
                            "name": "read_csv_to_dataframe",
                            "arguments": json.dumps({
                                "file_path": "/content/Beer_small_test.csv",
                                "column_name": "text",
                                "num_entries": 5
                            })
                        }
                    }
                }
            ]
        }

# Instantiate the client
client = MockGPT4oClient()

# Generate the response
response = client.chat(
    completions={
        "model": "gpt-4o",
        "messages": messages,
        "functions": tools,
        "function_call": {"name": "read_csv_to_dataframe"}
    }
)

# Process the response
if response["choices"][0]["message"].get("function_call"):
    function_call = response["choices"][0]["message"]["function_call"]
    function_name = function_call["name"]
    function_arguments = json.loads(function_call["arguments"])

    if function_name == "read_csv_to_dataframe":
        # Call the function
        result = read_csv_to_dataframe(
            file_path=function_arguments["file_path"],
            column_name=function_arguments["column_name"],
            num_entries=function_arguments["num_entries"]
        )

        # Check for errors
        if "error" in result:
            user_answer = f"I tried to read the file, but encountered an error: {result['error']}"
        else:
            # Extracted data
            extracted_data = result["data"]
            column_name = result["column_name"]

            # Craft a new prompt for summarization
            new_messages = [
                {"role": "system", "content": "You are an assistant that analyzes text data."},
                {"role": "user", "content": f"Please summarize the following data from the '{column_name}' column: {extracted_data}"}
            ]

            # Simulate the model summarizing the text
            summary_response = client.chat(
                completions={
                    "model": "gpt-4o",
                    "messages": new_messages
                }
            )

            # Mock summary response
            summary = (
                "The extracted entries discuss various aspects, including the flavor, texture, and uniqueness "
                "of different beers. Common themes include customer preferences and the distinct character of each beer."
            )

            user_answer = f"Summary of the '{column_name}' column:\n{summary}"

        # Print the response
        print(user_answer)
else:
    print("No function call found in the response.")